# Gate Fidelity Benchmarking

In [ ]:
import matplotlib.pyplot as plt
from equal1.benchmarking.device_characterization.gate_fidelity import Gate2qFidelity
import pandas as pd
import seaborn as sns
from rich import progress
import itertools

In [ ]:
hardware = {
    "hpc-conservative",
    "hpc-gamma1",
    "hpc-gamma2",
}
num_qubits = 16

## Two-Qubit Gates Fidelity

In [ ]:
gates_to_benchmark = ["cz"]

experiments = [
    Gate2qFidelity(device_name=hw, qubit_pair=(q1, q2), gate_to_benchmark=g)
    for hw in hardware
    for q1, q2 in itertools.combinations(range(num_qubits), 2)
    for g in gates_to_benchmark
]

In [ ]:
results = []


for experiment in progress.track(
    experiments,
    description="Running Experiments",
    auto_refresh=False,
):
    experiment.run(
        simulate=True,
        runtime_options={
            "backend": "TensorNet_MPS",
            "simulation_platform": "GPU",
        },
    )

    fidelity = experiment.analyse_results()

    results.append(
        {
            "device": experiment.device_name,
            "qubit_0": experiment.qubits_to_test[0],
            "qubit_1": experiment.qubits_to_test[1],
            "gate_name": experiment.gate_to_benchmark.name,
            "fidelity": fidelity,
        }
    )


In [ ]:
df = pd.DataFrame(results)
df["qubit_pair"] = df["qubit_0"].astype(str) + "-" + df["qubit_1"].astype(str)
df.to_csv("two_qubit_gate_fidelity_results.csv", index=False)

In [ ]:
for device in df["device"].unique():
    fig, ax = plt.subplots(figsize=(24, 6))
    sns.barplot(
        data=df[df["device"] == device],
        x="qubit_pair",
        hue="gate_name",
        y="fidelity",
        ax=ax,
    )
    ax.set_ylabel("Gate Fidelity")
    ax.set_xlabel("Qubit Pair")
    for container in ax.containers:
        ax.bar_label(container, fmt="%.5f")
    ax.set_title(f"Two Qubit Gate Fidelity on {device}")
    plt.show()